In [1]:
from torchHMM.model.discretized_HMM import DiscreteHMM, DISCRETIZATION_TECHNIQUES
from torchHMM.model.discretized_flow_HMM import FlowHMM

In [2]:
import numpy as np
import pandas as pd
import pickle as pkl

In [3]:
raise ArithmeticError

ArithmeticError: 

In [4]:
from hmmlearn.hmm import CategoricalHMM, GaussianHMM
from gensim.models import KeyedVectors


In [5]:
with open('clickstream_experiment/data/preprocessed_data/ClickStream_test_cleaned.pkl', 'rb') as f:
    cs = pkl.load(f)

In [6]:
item_ids = np.unique([e.item_id for s in cs.sessions for e in s.event_list])
item_ids = {item_ids[i]: i for i in range(len(item_ids))}

Xs = [[item_ids[e.item_id] for e in s.event_list[:-1]] for s in cs.sessions]
l = [len(y) for y in Xs]

X = np.concatenate(Xs).reshape(-1, 1)

target = [item_ids[s.event_list[-1].item_id] for s in cs.sessions]

In [7]:
X

array([[147834],
       [ 75421],
       [126066],
       ...,
       [ 35192],
       [ 35192],
       [ 35192]])

In [8]:
chmm = CategoricalHMM(10, n_iter=10, verbose=True)


In [9]:
%time chmm.fit(X, l)

         1   -51685633.1042             +nan
         2   -47028710.6338    +4656922.4704
         3   -46657692.7329     +371017.9009
         4   -46283643.3200     +374049.4129
         5   -45929378.6766     +354264.6433
         6   -45627183.0599     +302195.6167
         7   -45363721.2001     +263461.8598
         8   -45116873.8774     +246847.3226
         9   -44896584.2567     +220289.6207


CPU times: user 15min 39s, sys: 280 ms, total: 15min 40s
Wall time: 15min 40s


        10   -44697618.6723     +198965.5844


CategoricalHMM(n_components=10,
               random_state=RandomState(MT19937) at 0x7FA8F47C4A40,
               verbose=True)

In [10]:
l1 = chmm.score(np.array(X).reshape(-1, 1), l)
l2 = chmm.score(np.concatenate([[item_ids[e.item_id] for e in s.event_list] for s in cs.sessions]).reshape(-1, 1), np.array(l) + 1)
(l2 - l1) / len(l)

-inf

In [11]:
vectors = KeyedVectors.load(
    f"clickstream_experiment/data/preprocessed_data/vectors_train_100_10_5_8_cleaned.kv"
)
vecs = np.concatenate(
    [
        vectors.get_vector(k).reshape(1, -1)
        for k in list(vectors.key_to_index.keys())
    ]
)

In [12]:
unknown = vecs.mean(axis=0).reshape(1, -1)

def get_vec(e):
    try:
        return vectors.get_vector(e.item_id).reshape(1, -1)
    except:
        return unknown

In [13]:
Xs_c = [[get_vec(e) for e in s.event_list[:-1]] for s in cs.sessions]
l_c = [len(y) for y in X]

X_c = np.concatenate([a for as_ in Xs_c for a in as_], axis=0)

target_c = [get_vec(s.event_list[-1]) for s in cs.sessions]

In [14]:
X_c.shape

(4147529, 100)

In [15]:
w2vhmm = GaussianHMM(10, n_iter=10, verbose=True)

In [16]:
%time w2vhmm.fit(X_c, l_c)

/ziob/klaudia/miniconda3/envs/preproc/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
         1   755559764.0953             +nan
         2  1575918057.7016  +820358293.6063
         3  2721462104.5343 +1145544046.8327
         4  3571390503.1894  +849928398.6550
         5  3571878318.1528     +487814.9634
         6  3572395390.4214     +517072.2687
         7  3572792503.8208     +397113.3994
         8  3573174291.8044     +381787.9836
         9  3573489053.6890     +314761.8845
        10  3573712837.4805     +223783.7915
Some rows of transmat_ have zero sum because no transition from the state was ever observed.


CPU times: user 1h 54min 11s, sys: 52.6 s, total: 1h 55min 3s
Wall time: 1h 51min 50s


GaussianHMM(n_components=10, verbose=True)

In [17]:
l1c = w2vhmm.score(X_c, l_c)
l2c = w2vhmm.score(np.concatenate([get_vec(e) for s in cs.sessions for e in s.event_list]), np.array(l_c) + 1)
(l2c - l1c) / len(l_c)

ValueError: transmat_ rows must sum to 1 (got [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.])

In [18]:
w2v_dhmm = DiscreteHMM(n_components=10, no_nodes=512, l=100, optimizer="Adam", verbose=True)

In [19]:
%time w2vhmm.fit(X_c, l_c)

Even though the 'startprob_' attribute is set, it will be overwritten during initialization because 'init_params' contains 's'
Even though the 'transmat_' attribute is set, it will be overwritten during initialization because 'init_params' contains 't'
Even though the 'means_' attribute is set, it will be overwritten during initialization because 'init_params' contains 'm'
/ziob/klaudia/miniconda3/envs/preproc/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
Even though the 'covars_' attribute is set, it will be overwritten during initialization because 'init_params' contains 'c'
         1   803376936.7538             +nan
         2  1560064125.5770  +756687188.8232
         3  2642046429.7412 +1081982304.1642
         4  3571535878.5658  +929489448.8246
         5  3572054154.7288     +518276.1630
         6  357

CPU times: user 1h 57min 43s, sys: 50.6 s, total: 1h 58min 33s
Wall time: 1h 55min 43s


GaussianHMM(n_components=10, verbose=True)

In [20]:
l1c_d = w2v_dhmm.score(X_c, l_c)
l2c_d = w2v_dhmm.score(np.concatenate([get_vec(e) for s in cs.sessions for e in s.event_list]), np.array(l_c) + 1)
(l2c_d - l1c_d) / len(l_c)

NotFittedError: This DiscreteHMM instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.

In [21]:
w2v_fhmm = FlowHMM(n_components=10, no_nodes=512, l=100, optimizer="Adam", verbose=True)
%time w2v_fhmm.fit(X_c, l_c)
l1c_f = w2v_fhmm.score(X_c, l_c)
l2c_f = w2v_fhmm.score(np.concatenate([get_vec(e) for s in cs.sessions for e in s.event_list]), np.array(l_c) + 1)
(l2c_f - l1c_f) / len(l_c)

MemoryError: Unable to allocate 791. GiB for an array with shape (4147529, 100, 512) and data type float32

NotFittedError: This FlowHMM instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.